In [25]:
import os
import subprocess
import numpy as np
import pandas as pd
import random
import mplcursors
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import font_manager

In [26]:
def run_cpp_program(noise, nPose):
    result = subprocess.run(['../build/MyGTSAM', f'{noise}', f'{nPose}'], 
                            capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error running C++ program: {result.stderr}")
        return np.inf

    return result.stdout.strip().split('\n')

In [27]:
path = "/home/ali/Github/Kamal_Dataset/seq8"

content = pd.read_csv(os.path.join(path, "stereo/static_poses.csv"), header=None)
Data = content.values
x_GT = Data[:,0]
y_GT = Data[:,1]
z_GT = Data[:,2]
nPose = len(x_GT)

content = pd.read_csv(os.path.join(path, "encoder/encoder.csv"), header=None)
Data = content.values
delta = Data[:,0]
tsta = Data[:,1]
brk = Data[:,2]

# IK with Optimized Anchor Point Locations

In [28]:
r_Delta = 0.025
r_TSTA = 0.035
r_Break = 0.035
l_enc = np.column_stack((r_Delta*delta, r_TSTA*tsta, r_Break*brk))

rmse = []
mean_rmse = []
num = [n for n in range(5,nPose+1)]
for i in num:
    result = np.array(list(map(float, run_cpp_program(0.1, i))))
    result_pd = pd.DataFrame((result[:15]).reshape(1,-1))
    result_pd.columns = ['s', 'm0', 'n0', 'k0', 'h0', 'A1_x', 'A1_y', 'A2_x', 'A2_y', 'A3_x', 'A3_y', 'A4_x', 'A4_y', 'X0_x', 'X0_y']

    x_GT += result_pd['X0_x'].values - x_GT[0]
    y_GT += result_pd['X0_y'].values - y_GT[0]

    A1 = [result_pd['A1_x'].values, result_pd['A1_y'].values]
    A2 = [result_pd['A2_x'].values, result_pd['A2_y'].values]
    A3 = [result_pd['A3_x'].values, result_pd['A3_y'].values]
    A4 = [result_pd['A4_x'].values, result_pd['A4_y'].values]

    l_0 = np.column_stack((result_pd['m0'] + result_pd['n0'], result_pd['m0'] + 2*result_pd['k0'], result_pd['n0'] + 2*result_pd['h0']))
    m = np.sqrt((x_GT[:i+1] - A1[0])**2 + (y_GT[:i+1] - A1[1])**2)
    n = np.sqrt((x_GT[:i+1] - A2[0])**2 + (y_GT[:i+1] - A2[1])**2)
    k = np.sqrt((x_GT[:i+1] - A3[0])**2 + (y_GT[:i+1] - A3[1])**2)
    h = np.sqrt((x_GT[:i+1] - A4[0])**2 + (y_GT[:i+1] - A4[1])**2)
    l = np.column_stack((m + n, m + 2*k, n + 2*h))
    delta_l = l - l[0]

    error = delta_l - l_enc[:i+1]
    error_rmse = np.sqrt(np.mean(error**2, axis=0))
    rmse.append(error_rmse)
    mean_rmse.append(np.mean(error_rmse))

rmse = np.array(rmse)

# IK with Nominal Anchor Point Locations

In [29]:
# a = 1.2
# b = 1.9

# A1 = [0, b]
# A2 = [a, b]
# A3 = [0, 0]
# A4 = [a, 0]
# X0 = [0.605, 1.9 - 0.805]

# l_0 = np.column_stack((1.0 + 1.0, 1.0 + 2*1.25, 1.0 + 2*1.246))
# m = np.sqrt((x_GT - A1[0])**2 + (y_GT - A1[1])**2)
# n = np.sqrt((x_GT - A2[0])**2 + (y_GT - A2[1])**2)
# k = np.sqrt((x_GT - A3[0])**2 + (y_GT - A3[1])**2)
# h = np.sqrt((x_GT - A4[0])**2 + (y_GT - A4[1])**2)
# l = np.column_stack((m + n, m + 2*k, n + 2*h))
# delta_l = l - l[0]

# error_isaac = delta_l - l_enc

# EPE_nom = np.sqrt(np.mean(error_isaac**2, axis=1))

In [36]:
plt.close('all')

# plt.figure()
# %matplotlib qt
# plt.plot(rmse, color="tab:blue", linestyle='--', marker='o')
# mplcursors.cursor(multiple=True)
# plt.xlabel("Number of Static Pose", **csfont)
# plt.ylabel("RMSE (m)", **csfont)
# plt.grid()

# plt.figure()
# %matplotlib qt
# plt.scatter(result_pd[['A1_x', 'A2_x', 'A3_x', 'A4_x']], result_pd[['A1_y', 'A2_y', 'A3_y', 'A4_y']], color="tab:orange", label="Refined Anchor Points")
# plt.plot(x_GT, y_GT, color="tab:green", linestyle='--', marker='o', label="Ground Truth")
# plt.plot(np.concatenate((result_pd['X0_x'].values, result[15::2])), np.concatenate((result_pd['X0_y'].values, result[16::2])), color="tab:blue", linestyle='--', marker='o', label="Refined End-Effector Position")
# plt.xlabel("x (m)", fontsize=16)
# plt.ylabel("y (m)", fontsize=16)

# plt.tick_params(axis='both', which='major', labelsize=14)
# plt.tick_params(axis='both', which='minor', labelsize=12)

plt.figure()
%matplotlib qt
# plt.plot(num, rmse, label=['Cable 1 Error', 'Cable 2 Error', 'Cable 3 Error'], linestyle='--', marker='o')
plt.plot(num, rmse[:, 0], label='Cable 1 Error', color="tab:red",linestyle='--', marker='o', markersize=10)
plt.plot(num, rmse[:, 1], label='Cable 2 Error', color="tab:green",linestyle='--', marker='^', markersize=10)
plt.plot(num, rmse[:, 2], label='Cable 3 Error', color="tab:blue",linestyle='--', marker='s', markersize=10)
plt.plot(num, mean_rmse, label="Average Error", color="black", linestyle='--', marker='*', markersize=12)
mplcursors.cursor(multiple=True)
csfont = {'fontname':'Times New Roman', 'fontsize': 22}
plt.xlabel("Number of Position Sample", **csfont)
plt.ylabel("Error (m)", **csfont)
font = font_manager.FontProperties(family='Times New Roman',
                                   style='normal', size=18)
plt.legend(prop=font)
plt.grid(which='major', linestyle=':', linewidth='0.8', color='gray')

plt.tick_params(axis='both', which='major', labelsize=18, labelfontfamily='Times New Roman')